In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

from Tight_Binding.tight_binding.extract_parameter import input_data
from Tight_Binding.tight_binding.extract_parameter import extract_parameter
from Tight_Binding.tight_binding.hamiltonian import hamiltonian
from Tight_Binding.tight_binding.hamiltonian import multiple_hamiltonian

In [2]:
from Tight_Binding.tight_binding.hamiltonian import hamiltonian_v2
from Tight_Binding.tight_binding.hamiltonian import hamiltonian_v3

# Import Data

In [3]:
loc_parameter = "Data/Sr4Nb4O12/hr_files/sr4nb4o12_hr.dat"
loc_atom_position = "Data/Sr4Nb4O12/atomic_position.csv"
loc_orbital_index = "Data/Sr4Nb4O12/orbital_index.csv"

In [4]:
parameter = input_data(loc_parameter, loc_atom_position, loc_orbital_index, 
           a = 10.86909,
           b = 5.763864449563357*10.86909,
           c = 0.703931807842342*10.86909)

In [5]:
parameter.get_parameter_TB().head()

,X,Y,Z,A,B,Re,Im
0,-3,-3,-3,1,1,-0.000552,0.000000
1,-3,-3,-3,2,1,0.001963,0.000773
2,-3,-3,-3,3,1,0.000304,-0.000521
3,-3,-3,-3,4,1,0.000057,-0.000251
4,-3,-3,-3,5,1,0.000500,0.000224


In [6]:
input_hamiltonian = parameter.vec_lattice()
input_hamiltonian.head()

,X,Y,Z,A,B,Re,Im,Rx,Ry,Rz
0,-3,-3,-3,1,1,-0.000552,0.000000,-32.60727,-187.943884,-22.953295
1,-3,-3,-2,1,1,-0.000232,-0.000032,-32.60727,-187.943884,-15.302196
2,-3,-3,-1,1,1,-0.000085,0.000092,-32.60727,-187.943884,-7.651098
3,-3,-3,0,1,1,-0.001114,0.000000,-32.60727,-187.943884,0.000000
4,-3,-3,1,1,1,-0.000085,-0.000092,-32.60727,-187.943884,7.651098


In [7]:
len(input_hamiltonian)

5976432

# Hamiltonian

In [8]:
k_grid = np.ones(shape=(12,3))

In [9]:
start = time.time()
ham = hamiltonian(k_grid[0], input_hamiltonian)
print((time.time() - start), "s")

print(ham[3,13], ham[13,3])

Hamiltonian 1 K-point berhasil digenerate. 333.36360597610474 Detik
333.3769438266754 s
(-0.0652039559955847-0.0007899319126766398j) (-0.0652039559955847+0.0007899319126766415j)


In [10]:
start = time.time()
ham = hamiltonian_v2(k_grid[0], input_hamiltonian)
print((time.time() - start), "s")

print(ham[3,13], ham[13,3])

57.488518476486206 s
(-0.06520395599558465-0.0007899319126766331j) (-0.06520395599558462+0.0007899319126766354j)


In [11]:
start = time.time()
ham = hamiltonian_v3(k_grid[0], input_hamiltonian)
print((time.time() - start), "s")

print(ham[3,13], ham[13,3])

23.140385389328003 s
(-0.06520395599558465-0.0007899319126766331j) (-0.06520395599558462+0.0007899319126766354j)


# Multiple Hamiltonian

In [9]:
start = time.time()
ham = multiple_hamiltonian(k_grid, input_hamiltonian, hamiltonian_v2, num_process=3)
print((time.time() - start), "s")

print(ham[0,3,13], ham[0,13,3])

Using  3 Processes
257.11468982696533 s
(-0.06520395599558465-0.0007899319126766331j) (-0.06520395599558462+0.0007899319126766354j)


In [10]:
start = time.time()
ham = multiple_hamiltonian(k_grid, input_hamiltonian, hamiltonian_v3, num_process=3)
print((time.time() - start), "s")

print(ham[0,3,13], ham[0,13,3])

Using  3 Processes
110.1421594619751 s
(-0.06520395599558465-0.0007899319126766331j) (-0.06520395599558462+0.0007899319126766354j)
